In [1]:
import pandas as pd
#read_data
sales_order = pd.read_csv('in/tables/sales_order.csv')
sales_order_item = pd.read_csv('in/tables/sales_order_item.csv')
hu = pd.read_csv('in/tables/HU_postal.csv')
cz = pd.read_csv('in/tables/CZ_postal.csv')
sk = pd.read_csv('in/tables/SK_postal.csv')

#replace spaces in postal codes of SLovakia and Czech Republic
cz['postal_code'] = cz['postal_code'].astype(str).str.replace(' ', '')
sk['postal_code'] = sk['postal_code'].astype(str).str.replace(' ', '')
cz['postal_code'] = cz['postal_code'].astype(int)
sk['postal_code'] = sk['postal_code'].astype(int)
cz['postal_code'] = cz['postal_code'].astype(int)
#concat country with cities and postal codes
postal_all = pd.concat([cz, sk, hu], ignore_index=True)
#delete postal code duplicates
postal_all = postal_all.drop_duplicates('postal_code')

sales_order = sales_order.fillna(0)
sales_order['postal_code'] = sales_order['postal_code'].astype(int)

#add 0 on the beggining in Slovsakia postal code if it has 4 digits
sales_order.loc[sales_order['country_code'] == 'SK', 'postal_code'] = (
    sales_order.loc[sales_order['country_code'] == 'SK', 'postal_code']
    .astype(str)        
    .str.zfill(5)       
)
sales_order.loc[sales_order['country_code'] == 'SK']
sales_order['postal_code'] = sales_order['postal_code'].astype(int)

sales_order_m = sales_order.merge(sales_order_item, how='left', left_on = 'pk_sales_order', right_on = 'fk_sales_order')
sales_order_city = sales_order_m.merge(postal_all, how = 'left', on=['postal_code', 'country_code'])
sales_order_city.to_csv('out/tables/city_agg.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'in/tables/sales_order.csv'

In [ ]:
import pandas as pd
#read data
sales_order = pd.read_csv('in/tables/sales_order.csv')
sales_order_item = pd.read_csv('in/tables/sales_order_item.csv')

#join datasets
sales_order_item['fk_sales_order'].value_counts()
sales_order_m = sales_order.merge(sales_order_item, how='left', left_on = 'pk_sales_order', right_on = 'fk_sales_order')

#transform cretaed_at to date 
sales_order_m['created_at'] = pd.to_datetime(sales_order_m['created_at'])
#extract month and year
sales_order_m['month'] = sales_order_m['created_at'].dt.month
sales_order_m['year'] = sales_order_m['created_at'].dt.year

#aggregate sales by price and cost
sales_order_m['sold_qty'] = sales_order_m['sold_qty'].astype(int)
sales_order_m['sell_price'] = (sales_order_m['product_price_local_currency'] * sales_order_m['currency_rate'] * sales_order_m['sold_qty']) 
sales_order_m['cost'] = sales_order_m['product_cost_eur']*sales_order_m['sold_qty']
monthly = sales_order_m.groupby(['year', 'month', 'fk_item']).agg(
    total_sell = ('sell_price', 'sum'),
    total_cost = ('cost', 'sum')
).reset_index()

#calculate avg monthly margin
monthly['monthly_margin'] = ((monthly['total_sell'] - monthly['total_cost']) / monthly['total_sell']).round(4)
monthly = monthly[monthly['total_sell'] != 0]
monthly.to_csv('out/tables/monthly_margin.csv', index=False)

In [ ]:
import pandas as pd
sales_order = pd.read_csv('in/tables/sales_order.csv')
sales_order_item = pd.read_csv('in/tables/sales_order_item.csv')
sales_order_m = sales_order.merge(sales_order_item, how='left', left_on = 'pk_sales_order', right_on = 'fk_sales_order')
sales_order_m['sell_price'] = (sales_order_m['product_price_local_currency'] * sales_order_m['currency_rate'] * sales_order_m['sold_qty']) 
from itertools import combinations

#create dataset without gift
no_prize = sales_order_m[sales_order_m['sell_price'] != 0].copy()
no_prize['fk_item'] = no_prize['fk_item'].astype('string')
no_prize = no_prize.dropna(subset=['fk_item'])

#aggregate by order and create list of items
no_prize_agg = no_prize.groupby('pk_sales_order')['fk_item'].agg(list).reset_index()
#combine in every possible pairs + filter if order has inly one item
pairs = no_prize_agg['fk_item'].apply(lambda items: list(combinations(items, 2)) if len(items) >= 2 else []).explode().dropna()

#top 10 pairs
pair_counts = pairs.value_counts().reset_index(name='count').rename(columns={'index':'pair'})
total_orders = (no_prize_agg['fk_item'].str.len() >= 2).sum()
pair_counts['percantage(%)'] = ((pair_counts['count'] / total_orders) * 100).round(2)
top10 = pair_counts.sort_values('count', ascending=False).head(10)

top10.to_csv('out/tables/top10_pairs.csv', index=False)